# Robustness Checks

In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import matplotlib as mpl 

mpl.rcParams['figure.dpi']= 200
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

from rpy2.robjects import pandas2ri
pandas2ri.activate()

%reload_ext rpy2.ipython

In [3]:
%%R

library('lme4')
library('margins')
library("performance")
library('tidyverse')

getICCs <- function(m, type = 'poisson'){
  var_s <- as.numeric(getME(m, "theta")[1]^2) # seller level variance
  var_w <- as.numeric(getME(m, "theta")[2]^2) # week level variance
  
  if(type == 'poisson'){
    lambda = .139
    alpha <- log(1 + 1/lambda)
  }
  if(type == 'binomial'){
    alpha <- (pi^2) / 3
  }
  
  icc <- list(
    s = (var_s)/ (var_s + var_w + alpha),
    w = (var_w)/ (var_s + var_w + alpha),
    t = (var_s + var_w)/ (var_s + var_w + alpha),
    a = (alpha)/ (var_s + var_w + alpha)
  )
  
  return(icc)
}

sessionInfo()

R[write to console]: Loading required package: Matrix

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

R[write to console]: ✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()



R version 3.6.1 (2019-07-05)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS Catalina 10.15.4

Matrix products: default
BLAS:   /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.6/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] forcats_0.4.0     stringr_1.4.0     dplyr_0.8.3       purrr_0.3.2      
 [5] readr_1.3.1       tidyr_1.0.0       tibble_2.1.3      ggplot2_3.2.1    
 [9] tidyverse_1.2.1   performance_0.4.5 margins_0.3.23    lme4_1.1-23      
[13] Matrix_1.2-17    

loaded via a namespace (and not attached):
 [1] statmod_1.4.34    tidyselect_0.2.5  splines_3.6.1     haven_2.1.1      
 [5] lattice_0.20-38   colorspace_1.4-1  vctrs_0.2.0       generics_0.0.2   
 [9] rlang_0.4.2       nloptr_1.2.1   

## Load Data

In [4]:
MAINDIR = os.getcwd().rsplit('/', 1)[0]
file = os.path.join(MAINDIR, 'data/analysis', 'vendor_item_period.csv')

In [5]:
df = pd.read_csv(file, index_col = False)
df = df.drop(df.columns[0], axis=1)

In [6]:
me = pd.read_pickle(os.path.join(MAINDIR, 'data/analysis', 'market_entry.pickle'))

## Robustness of ARF

In our paper we used liberal cut of values, here we provide robustness checks for stricter cut of criteria

In [7]:
# setting cut of values
t_cut = me['delta_t'].quantile(1/3)
p_cut = me['price_usd'].quantile(1/3)
s_cut = me.groupby('vendor')['delta_t'].std().quantile(1/3)


# ARF 1
arf1 = me\
    .groupby('vendor')\
    .apply(lambda x: (x['delta_t'].mean() < t_cut\
                      and (x['price_usd'].mean() < p_cut)\
                      and (x['delta_t'].std() < s_cut)))\
    .rename('arf_025')

# setting cut of values
t_cut = me['delta_t'].quantile(2/5)
p_cut = me['price_usd'].quantile(2/5)
s_cut = me.groupby('vendor')['delta_t'].std().quantile(2/5)


# ARF 1
arf2 = me\
    .groupby('vendor')\
    .apply(lambda x: (x['delta_t'].mean() < t_cut\
                      and (x['price_usd'].mean() < p_cut)\
                      and (x['delta_t'].std() < s_cut)))\
    .rename('arf_041')

arf = pd.concat([arf1, arf2], axis = 1).reset_index()\
    .assign(vendor = lambda x: x['vendor'].factorize()[0] + 1)

## Robustness of ARM

In our paper we used global cut of values to determine cases of artificial growth. However, some sellers might aswell have show fast, static and cheap growth by definition. Therefore we perform robustness checks were we includes a stricter definition of ARM, including internally defined cut-off values. This assumes that ARM is a strategy that is used at some time-point but not all the time during market activity. However, it assures that the growth of reputation is relatively fast, static and cheap for sellers themselves. 

In [8]:
# make ARM variable
t_cut_per_seller = df.groupby('vendor')['delta_t_m'].quantile(.5).to_dict()
s_cut_per_seller = df.groupby('vendor')['delta_t_s'].quantile(.5).to_dict()
p_cut_per_seller = df.groupby('vendor')['price_usd_m'].quantile(.5).to_dict()
t_cut = df['delta_t_m'].quantile(1/2)
p_cut = df['price_usd_m'].quantile(1/2)
s_cut = df['delta_t_s'].quantile(1/2)

arm = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller))\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().rename('arm_i').reset_index()

arm['arm_t'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_p'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            p_low   = lambda x: x['price_usd_m'] < p_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 'p_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_tp'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low', 'p_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

arm['arm_a'] = df\
    .assign(t_low_s = lambda x: x['delta_t_m'] < x['vendor'].map(t_cut_per_seller),
            p_low_s = lambda x: x['price_usd_m'] < x['vendor'].map(p_cut_per_seller),
            s_low_s = lambda x: x['delta_t_s'] < x['vendor'].map(s_cut_per_seller),
            t_low   = lambda x: x['delta_t_m'] < t_cut,
            p_low   = lambda x: x['price_usd_m'] < p_cut,
            s_low   = lambda x: x['delta_t_s'] < s_cut)\
    .assign(arm = lambda x: (x[['t_low_s', 'p_low_s', 's_low_s', 't_low', 'p_low', 's_low']].all(axis = 1)))\
    .groupby(['vendor', 'w'])['arm'].max().reset_index()['arm']

# Shift Dependent variables with one week 
for col in ['arm_tp', 'arm_p', 'arm_t', 'arm_a', 'arm_i']: 
    arm['{0}_shift'.format(col)] = arm\
        .groupby('vendor')\
        .apply(lambda x: x[col].shift(1))\
        .reset_index(drop = True)

In [9]:
# import person week file
df = pd.read_pickle(os.path.join(MAINDIR, 'data/analysis', 'vendor_week.pickle'))
df = df.merge(df.groupby('vendor')['me_min'].min().rename('me'), on = 'vendor')
df = df.rename(columns= {
    'international_shipment_count_w': 'int_shipment_count_w', 
    'international_shipment': 'int_shipment'})

In [10]:
df = df.merge(arm, on = ['vendor', 'w']).merge(arf, on = ['vendor'])

In [11]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()

# normalize variables
skewed_variables = [
    'neg_count_min', 'pos_count_min', 'neg_count_w', 
    'neg_count_w_shift', 'pos_count_w', 'pos_count_w_shift', 'sales_volume_w', 
    'item_count_w', 'int_shipment_count_w']

bc_vars = pd.DataFrame(
    pt.fit_transform(df[skewed_variables]),
    columns = ['bc_' + var for var in skewed_variables])

c_vars = df[skewed_variables].add_prefix('c_')

dummy_vars = df.assign(
        items   = pd.qcut(df['item_count'], 3,  labels=["low", "medium", "high"]),
        items_w = pd.qcut(df['item_count_w'], 3,  labels=["low", "medium", "high"]),
        sales   = pd.qcut(df['cum_count'], 2,  labels=["low", "high"]),
        sales_w = pd.qcut(df['sales_volume_w'], 2,  labels=["low", "high"]))\
    .loc[:,['items', 'items_w', 'sales', 'sales_w']]

n_vars = df[['cum_count', 'sales_volume_w', 'item_count', 
             'item_count_w', 'me', 'int_shipment_count_w',
             'empty_stock_last_week_count_w', 'empty_stock_last_week_maxw']].rename({'cum_count': 'sales_volume'})

bool_vars = df[
    ['arf', 'arf_025', 'arf_041', 'arm_maxw', 'arm_maxw_shift', 'arm_i', 'arm_a', 'arm_t', 'arm_p', 'arm_tp',
     'int_shipment', 'has_price_drop', 'has_price_drop_shift']].astype('int')

lev_vars =  df[['vendor','w', 'me_min']]

data = pd.concat([lev_vars, bool_vars, bc_vars, c_vars, dummy_vars, n_vars], axis = 1)

In [12]:
%%R -i data

data['me_'] <- scale(data$me)[,1]
data['w_'] <- scale(data$w)[,1]
data['me_2'] <- scale(data$me^2)[,1]
data['w_2'] <- scale(data$w^2)[,1]

variables = c(
    'bc_pos_count_w', 'bc_item_count_w', 'bc_int_shipment_count_w',
    'bc_neg_count_w', 'c_neg_count_w', 'c_pos_count_w')

for (var in variables){
  for (vendor in unique(data$vendor)){
    # calculate mean and deviances
    x = data[data$vendor == vendor, var]
    m = mean(x)
    dev = x - m
    
    # concat information to dataframe
    data[data$vendor == vendor, paste(var, "m", sep=".")] <- m
    data[data$vendor == vendor, paste(var, "dev", sep=".")] <- dev
  }
}

Optimizer <- glmerControl(optimizer = "bobyqa",
                          optCtrl = list(maxfun=2e5))

## ARF
### Stricter criteria

In [13]:
%%R
model = glmer(c_neg_count_w_shift ~ arf_041
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: 
c_neg_count_w_shift ~ arf_041 + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
147342.3 147454.7 -73657.1 147314.3    22763 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-13.397  -0.946  -0.414  -0.024  61.057 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 1.9283   1.3886  
 w      (Intercept) 0.8217   0.9065  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -0.414916   0.149512  -2.775 0.005518 ** 
arf_041                      0.697095   0.318514   2.189 0.028627 * 

In [14]:
%%R
model = glmer(c_neg_count_w_shift ~ arf_025
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = poisson,
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: poisson  ( log )
Formula: 
c_neg_count_w_shift ~ arf_025 + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
147346.9 147459.4 -73659.5 147318.9    22763 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-13.397  -0.946  -0.414  -0.024  61.055 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 1.9401   1.3929  
 w      (Intercept) 0.8218   0.9065  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -0.396404   0.149267  -2.656 0.007915 ** 
arf_025                      0.067149   0.427403   0.157 0.875158   

In [15]:
%%R
model = glmer(arm_a ~ arf_041
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_041
      + bc_neg_count_w.dev:arf_041
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
arm_a ~ arf_041 + bc_neg_count_w.m + bc_neg_count_w.dev + bc_neg_count_w.m:arf_041 +  
    bc_neg_count_w.dev:arf_041 + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
  7587.9   7732.5  -3775.9   7551.9    22759 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.591 -0.119 -0.025 -0.003 44.680 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 2.05222  1.4326  
 w      (Intercept) 0.03307  0.1819  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -6.229908   0.164586 -37.852 

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




In [16]:
%%R
model = glmer(arm_a ~ arf_025
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_025
      + bc_neg_count_w.dev:arf_025
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
arm_a ~ arf_025 + bc_neg_count_w.m + bc_neg_count_w.dev + bc_neg_count_w.m:arf_025 +  
    bc_neg_count_w.dev:arf_025 + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
  7587.1   7731.7  -3775.6   7551.1    22759 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.581 -0.119 -0.025 -0.003 44.033 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 2.05447  1.4333  
 w      (Intercept) 0.03321  0.1822  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -6.237756   0.164687 -37.876 

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




In [17]:
%%R
model = glmer(has_price_drop_shift ~ arf_041
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_041
      + bc_neg_count_w.dev:arf_041
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
has_price_drop_shift ~ arf_041 + bc_neg_count_w.m + bc_neg_count_w.dev +  
    bc_neg_count_w.m:arf_041 + bc_neg_count_w.dev:arf_041 + bc_pos_count_w.m +  
    bc_pos_count_w.dev + bc_item_count_w.m + bc_item_count_w.dev +  
    bc_int_shipment_count_w.m + bc_int_shipment_count_w.dev +  
    me_ + me_2 + w_ + w_2 + (1 | vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
 13677.9  13822.5  -6821.0  13641.9    22759 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.0724 -0.3658 -0.2681 -0.1911  9.6106 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 0.000393 0.01983 
 w      (Intercept) 0.434092 0.65886 
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.5

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




In [18]:
%%R
model = glmer(has_price_drop_shift ~ arf_025
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf_025
      + bc_neg_count_w.dev:arf_025
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

R[write to console]: boundary (singular) fit: see ?isSingular



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
has_price_drop_shift ~ arf_025 + bc_neg_count_w.m + bc_neg_count_w.dev +  
    bc_neg_count_w.m:arf_025 + bc_neg_count_w.dev:arf_025 + bc_pos_count_w.m +  
    bc_pos_count_w.dev + bc_item_count_w.m + bc_item_count_w.dev +  
    bc_int_shipment_count_w.m + bc_int_shipment_count_w.dev +  
    me_ + me_2 + w_ + w_2 + (1 | vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
 13678.0  13822.6  -6821.0  13642.0    22759 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.0743 -0.3659 -0.2681 -0.1913  9.6305 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 0.0000   0.0000  
 w      (Intercept) 0.4341   0.6589  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -2.5

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




convergence code: 0
boundary (singular) fit: see ?isSingular



## ARM
### Internal and external cut-offs

In [19]:
%%R
model = glmer(arm_a ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
arm_a ~ arf + bc_neg_count_w.m + bc_neg_count_w.dev + bc_neg_count_w.m:arf +  
    bc_neg_count_w.dev:arf + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
  7585.8   7730.4  -3774.9   7549.8    22759 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-3.540 -0.119 -0.026 -0.003 45.636 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 2.02745  1.4239  
 w      (Intercept) 0.03264  0.1807  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -6.26886    0.16752 -37.421  < 2e-16 ***
a

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




In [20]:
%%R
model = glmer(arm_p ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
arm_p ~ arf + bc_neg_count_w.m + bc_neg_count_w.dev + bc_neg_count_w.m:arf +  
    bc_neg_count_w.dev:arf + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
 19546.5  19691.1  -9755.3  19510.5    22759 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-12.5016  -0.4652  -0.1050   0.4970  22.8718 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 2.11095  1.4529  
 w      (Intercept) 0.02125  0.1458  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -1.663685   0.070138 

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




In [21]:
%%R
model = glmer(arm_tp ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
arm_tp ~ arf + bc_neg_count_w.m + bc_neg_count_w.dev + bc_neg_count_w.m:arf +  
    bc_neg_count_w.dev:arf + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
  8341.0   8485.7  -4152.5   8305.0    22759 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-5.269 -0.117 -0.023 -0.002 52.126 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 2.78330  1.6683  
 w      (Intercept) 0.05737  0.2395  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -6.1663725  0.1710118 -36.058  < 2e-16 

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




### Optimized operationalization of ARM
- excluding cutoff for overall selling price
    - this assumes that sellers definition of cheap is internally defined
- excluding cutoff for overall standard deviation of delta time
    - this assumes that the staticity of reputation inflation is internally defined

In [22]:
%%R
model = glmer(arm_t ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
arm_t ~ arf + bc_neg_count_w.m + bc_neg_count_w.dev + bc_neg_count_w.m:arf +  
    bc_neg_count_w.dev:arf + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
  8585.7   8730.3  -4274.9   8549.7    22759 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-5.602 -0.129 -0.025 -0.002 65.954 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 1.90573  1.3805  
 w      (Intercept) 0.06567  0.2563  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -5.734035   0.149678 -38.309  < 2e-16 ***

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




In [23]:
%%R
model = glmer(arm_i ~ arf
      + bc_neg_count_w.m
      + bc_neg_count_w.dev
      + bc_neg_count_w.m:arf
      + bc_neg_count_w.dev:arf
      + bc_pos_count_w.m
      + bc_pos_count_w.dev
      + bc_item_count_w.m
      + bc_item_count_w.dev
      + bc_int_shipment_count_w.m
      + bc_int_shipment_count_w.dev
      + me_ + me_2 
      + w_ + w_2 
      + (1 | vendor) + (1 | w),
      data, 
      family = binomial, 
      control = Optimizer)

summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
arm_i ~ arf + bc_neg_count_w.m + bc_neg_count_w.dev + bc_neg_count_w.m:arf +  
    bc_neg_count_w.dev:arf + bc_pos_count_w.m + bc_pos_count_w.dev +  
    bc_item_count_w.m + bc_item_count_w.dev + bc_int_shipment_count_w.m +  
    bc_int_shipment_count_w.dev + me_ + me_2 + w_ + w_2 + (1 |  
    vendor) + (1 | w)
   Data: data
Control: Optimizer

     AIC      BIC   logLik deviance df.resid 
 20489.6  20634.2 -10226.8  20453.6    22759 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-35.267  -0.508  -0.095   0.535  39.719 

Random effects:
 Groups Name        Variance Std.Dev.
 vendor (Intercept) 1.20699  1.0986  
 w      (Intercept) 0.02409  0.1552  
Number of obs: 22777, groups:  vendor, 902; w, 50

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -1.165494   0.057155 -20.392  <

R[write to console]: 
Correlation matrix not shown by default, as p = 16 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it


